<a href="https://colab.research.google.com/github/saileepanchbhai/Advance-Machine-Learning-Lab/blob/main/Market_Basket_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing libraries
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Online Retail.xlsx to Online Retail.xlsx
User uploaded file "Online Retail.xlsx" with length 23715344 bytes


In [ ]:
df=pd.read_excel('Online Retail.xlsx') # Read the Excel file 'Online Retail.xlsx' and store it in a DataFrame named df
df.head() # Display the first 5 rows of the dataset

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
df.shape # Display the number of rows and columns in the dataset

(541909, 8)

In [ ]:
df.dtypes # Shows data type of each column

,0
InvoiceNo,object
StockCode,object
Description,object
Quantity,int64
InvoiceDate,datetime64[ns]
UnitPrice,float64
CustomerID,float64
Country,object


In [ ]:
df.isnull().sum()  # Detects missing values (True/False)
                   # Counts how many missing values are present in each column

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


In [ ]:
# Remove rows where the 'Description' column has missing (NaN) values
# axis=0 → operate on rows
# subset=['Description'] → check missing values only in Description column
# inplace=True → make changes directly in the original DataFrame
df.dropna(axis=0, subset=['Description'], inplace=True)

In [ ]:
df.isnull().sum()# Detects missing values (True/False)
                   # Counts how many missing values are present in each column

,0
InvoiceNo,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,133626
Country,0


In [ ]:
# Import regular expression (re) module
# Used for cleaning and formatting text data
import re
# Clean the 'Description' column
df['Description'] = df['Description'].astype('str').apply(lambda x: re.sub(r'\s+', ' ', x).strip())
    # Convert values to string type to avoid errors
    # Use regular expression to:
    # 1. Replace multiple spaces (\s+) with a single space
    # 2. Remove leading and trailing spaces using strip()

In [ ]:
# Filter transactions where InvoiceNo starts with 'C'
# 'C' indicates cancelled transactions in the Online Retail dataset

df[df['InvoiceNo'].astype(str).str.startswith('C')].sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
383069,C569995,23267,SET OF 4 SANTA PLACE SETTINGS,-48,2011-10-06 20:36:00,1.25,14557.0,United Kingdom
177995,C552212,23126,FELTCRAFT GIRL AMELIE KIT,-1,2011-05-06 15:08:00,4.95,17422.0,United Kingdom
77928,C542793,22891,TEA FOR ONE POLKADOT,-6,2011-02-01 10:52:00,4.25,14113.0,United Kingdom
29944,C538833,22383,LUNCH BAG SUKI DESIGN,-1,2010-12-14 13:05:00,1.45,14680.0,United Kingdom
501790,C578832,22867,HAND WARMER BIRD DESIGN,-1,2011-11-25 15:18:00,2.10,13069.0,United Kingdom
343174,C566941,23206,LUNCH BAG APPLE DESIGN,-1,2011-09-15 16:14:00,1.65,13136.0,United Kingdom
313378,C564488,23087,ZINC HEART T-LIGHT HOLDER,-15,2011-08-25 14:12:00,1.25,16859.0,United Kingdom
88059,C543755,21591,COSY HOUR CIGAR BOX MATCHES,-24,2011-02-11 14:48:00,1.25,14304.0,United Kingdom
38941,C539602,21231,SWEETHEART CERAMIC TRINKET BOX,-4,2010-12-20 14:02:00,1.25,13369.0,United Kingdom
352797,C567690,22659,LUNCH BOX I LOVE LONDON,-7,2011-09-21 17:01:00,1.95,15810.0,United Kingdom


In [ ]:
# Convert InvoiceNo column to string type
# This ensures we can perform string operations like searching for 'C'
df['InvoiceNo']=df['InvoiceNo'].astype('str')
# Remove cancelled transactions
# In this dataset, Invoice numbers containing 'C' indicate cancellations
# ~ (NOT operator) keeps only rows that do NOT contain 'C'
df=df[~df['InvoiceNo'].str.contains('C')]
# Display the updated shape of the dataset
# This shows how many rows and columns remain after removing cancellations
print(df.shape)

(531167, 8)


In [ ]:
# Count the number of unique countries in the dataset
# nunique() returns the total distinct values in the 'Country' column
df['Country'].nunique()

# Display the top countries with their relative frequency (percentage)
# value_counts(normalize=True) gives proportion instead of raw count
# head() shows top 5 countries
df['Country'].value_counts(normalize=True).head()

,proportion
Country,
United Kingdom,0.915283
Germany,0.017023
France,0.015829
EIRE,0.014862
Spain,0.004678


In [ ]:
# Count the number of unique countries in the dataset
# nunique() returns the total distinct values in the 'Country' column
df['Country'].nunique()
df['Country'].value_counts()

,count
Country,
United Kingdom,486168
Germany,9042
France,8408
EIRE,7894
Spain,2485
Netherlands,2363
Belgium,2031
Switzerland,1967
Portugal,1501


In [ ]:
grouped_df=(df[df['Country']=="Germany"] # Create transaction matrix for Germany only
            .groupby(['InvoiceNo','Description'])# Group by InvoiceNo and Description (each product in each invoice)
            .size()     # Count occurrences of each product per invoice
            .unstack() # Convert product names into columns (pivot format)
                        # Each row = one invoice
                        # Each column = one product
            .fillna(0) # Replace NaN values with 0 (means product not purchased in that invoice)
            .reset_index() # Reset index to make InvoiceNo a normal column temporarily
            .set_index('InvoiceNo')) # Set InvoiceNo again as index
print(grouped_df.shape) # Display shape of the transaction matrix
grouped_df.head() # Show first 5 rows

(457, 1693)


Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,...,YULETIDE IMAGES GIFT WRAP SET,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC HEART T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC STAR T-LIGHT HOLDER,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Define a custom encoding function
# This function converts quantity values into binary format (0 or 1)
# Required because Apriori works on binary transaction data
def custom_encoding(x):
  if x <=0:   # If quantity is 0 or negative → item not purchased
    return 0
  elif x==1:     # If quantity is 1 → item purchased
    return 1
  else:
    return 1   # If quantity is more than 1 → still mark as purchased (1)

# Apply the custom encoding function to entire transaction matrix
# Converts all product quantities into 0/1 format
grouped_df = grouped_df.applymap(custom_encoding)

# Replace any remaining NaN values with 0
# Ensures clean dataset before applying Apriori
grouped_df.fillna(0, inplace=True)

# Remove 'POSTAGE' column if it exists
# POSTAGE is not an actual product — it may distort association rules
# Check first to avoid KeyError
if 'POSTAGE' in grouped_df.columns:
    grouped_df.drop('POSTAGE', inplace=True, axis=1)


In [ ]:
# Apply Apriori algorithm to the transaction dataset (grouped_df)

# min_support = 0.05
# Means an itemset must appear in at least 5% of total transactions
# to be considered frequent

# use_colnames = True
# Shows actual product names instead of column index numbers
frequent_itemsets = apriori(grouped_df, min_support=0.05, use_colnames=True)

In [ ]:
# Generate association rules from the frequent itemsets

# metric = "confidence"
# Rules will be evaluated based on confidence value

# min_threshold = 0.6
# Only rules with confidence ≥ 60% will be generated
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

In [ ]:
# Sort the generated association rules based on Lift value

# by='lift' → Sort using lift metric
# ascending=False → Highest lift values first (strongest associations on top)
rules.sort_values(by='lift', ascending=False)



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(RED RETROSPOT CHARLOTTE BAG),(WOODLAND CHARLOTTE BAG),0.070022,0.126915,0.059081,0.843750,6.648168,1.0,0.050194,5.587746,0.913551,0.428571,0.821037,0.654634
1,(ROUND SNACK BOXES SET OF 4 FRUITS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.157549,0.245077,0.131291,0.833333,3.400298,1.0,0.092679,4.529540,0.837922,0.483871,0.779227,0.684524
2,(SPACEBOY LUNCH BOX),(ROUND SNACK BOXES SET OF4 WOODLAND),0.102845,0.245077,0.070022,0.680851,2.778116,1.0,0.044817,2.365427,0.713415,0.251969,0.577243,0.483283


In [ ]:
# Example: For United Kingdom
grouped_df = (
    df[df['Country'] == "United Kingdom"]
    .groupby(['InvoiceNo', 'Description'])
    .size()
    .unstack()
    .fillna(0)
)

# Convert to binary
grouped_df = grouped_df.applymap(lambda x: 1 if x > 0 else 0)

# Apply Apriori
frequent_itemsets = apriori(grouped_df,
                            min_support=0.05,
                            use_colnames=True)

# Generate rules
rules = association_rules(frequent_itemsets,
                          metric="confidence",
                          min_threshold=0.6)

# Sort by lift
rules = rules.sort_values(by='lift', ascending=False)

rules.head()


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
